# API: Particles and Pushers

Struphy provides a convenient interface to [particle data structures](https://struphy.pages.mpcdf.de/struphy/tutorials/tutorial_06_data_structures.html#PIC-data-structures) and [particle pushers](https://struphy.pages.mpcdf.de/struphy/sections/subsections/pic_base.html#module-struphy.pic.pushing.pusher) (ODE solvers) for trying out new particle schemes.
Check out the documentation for more details.

In [ ]:
from struphy.pic.particles import Particles3D

particles = Particles3D(name='ions', 
                        Np=10,
                        bc=['periodic']*3,
                        loading='pseudo_random',
                        )

particles.draw_markers()

In [ ]:
particles.markers

In [3]:
from struphy.pic.pushing.pusher import Pusher, ButcherTableau
from struphy.pic.pushing.pusher_kernels import push_random_diffusion_stage
from numpy import random, array

noise = array(particles.markers[:, :3])
diffusion_coeff = 1.
butcher = ButcherTableau('forward_euler')

args_kernel = (noise,
               diffusion_coeff,
               butcher.a,
               butcher.b,
               butcher.c)

pusher = Pusher(particles, 
                push_random_diffusion_stage, 
                args_kernel,
                particles.domain.args_domain,
                alpha_in_kernel=1.)